In [1]:
from SBM_attributed import *
from divergences import *
import numpy as np
import networkx as nx
from scipy.optimize import minimize,LinearConstraint
from sklearn.metrics import rand_score, calinski_harabasz_score, mutual_info_score, accuracy_score

In [2]:
P = np.array([[0.8, 0.2, 0.3],[0.2, 0.7, 0.4],[0.3, 0.4, 0.6]])
c = 3
n = 100
N = c*n
delta = 1
d = 2
dim = c*d
_,A,X = generate_benchmark(P,c,delta=delta,n=n,dim=d,sample_along_direction=True)
B = np.eye(c)
W = np.zeros(N*c).reshape((N,c))
indexes = np.random.randint(0,c,size=N)
for index,col in enumerate(indexes):
    W[index,col] = 1
mu = np.random.normal(size=(c,dim))

In [3]:
phi_net = get_phi("euclidean",elementwise=True)
phi_data = get_phi("euclidean")

In [4]:
### Generate benchmark
P = np.array([[0.8, 0.2, 0.3],[0.2, 0.7, 0.4],[0.3, 0.4, 0.6]])
c = 3
n = 100
N = c*n
delta = 10
d = 2
dim = c*d
_,A,X = generate_benchmark(P,c,delta=delta,n=n,dim=d,sample_along_direction=True)

In [5]:
### initialize variables
B = np.eye(c)
W = np.zeros(N*c).reshape((N,c))
indexes = np.random.randint(0,c,size=N)
for index,col in enumerate(indexes):
    W[index,col] = 1
mu = np.random.normal(size=(c,dim))

In [6]:
def make_divergence_matrix(phi_net,phi_data,A,X,W,B,mu):
    net_divergence = np.sum(pairwise_net_bregman(A,W@B@W.T,phi_net), axis=1)
    data_divergence = np.sum(np.multiply(W, pairwise_data_bregman(X, mu, phi_data)),axis=1)
    return np.stack((net_divergence,data_divergence),axis=-1)

In [7]:
model_args = {}
model_args["_c"] = c
model_args["_N"] = N
model_args["_dim"] = dim
model_args["_phi_net"] = phi_net
model_args["_phi_data"] = phi_data
model_args["_norm"] = -1
model_args["_A"] = A
model_args["_X"] = X
def obj_func(params,model_args,other_args=None):
    c = model_args["_c"]
    N = model_args["_N"]
    dim = model_args["_dim"]
    phi_net = model_args["_phi_net"]
    phi_data = model_args["_phi_data"]
    norm = model_args["_norm"]
    A = model_args["_A"]
    X = model_args["_X"]
    W = params[:N*c].reshape((N, c))
    B = params[N*c:N*c+c*c].reshape((c, c))
    mu = params[N*c+c*c:].reshape((c,dim))
    K = make_divergence_matrix(phi_net,phi_data,A,X,W,B,mu)
    return np.sum(np.linalg.norm(K,ord=norm,axis=1))

In [8]:
#sum of W matrix row wise is one 
def make_constraints(N,c,dim):
    C = np.block([np.kron(np.eye(N,dtype=int),np.ones(c)),np.zeros((N,c*c + c*dim))]) 
    return LinearConstraint(C,np.ones(N),np.ones(N))
def make_bounds(N,c,dim):
    W_B_bounds = [(0,1)]*(N*c + c*c)
    mu_bounds = [(-np.inf,np.inf)]*(c*dim)
    return np.vstack([W_B_bounds,mu_bounds])
bounds = make_bounds(N,c,dim)
constraints = make_constraints(N,c,dim)

In [9]:
for _ in range(4):
    x0 = np.hstack([W.flatten(),B.flatten(),mu.flatten()])
    res = minimize(    
                    fun=obj_func,
                    x0=x0,
                    args=model_args,
                    constraints=constraints,
                    bounds=bounds,
                    method="SLSQP",
                    options={"maxiter":20}
                  )
    model_args["_norm"] *= 2
    W = res.x[:N*c].reshape((N, c))
    B = res.x[N*c:N*c+c*c].reshape((c, c))
    mu = res.x[N*c+c*c:].reshape((c,dim))
    pred_labels = np.argmax(W,axis=1)
    true_labels = [0]*n + [1]*n + [2]*n
    print(rand_score(true_labels, pred_labels),mutual_info_score(true_labels, pred_labels))


KeyboardInterrupt

